In [1]:
from sklearn.pipeline import Pipeline
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_curve, \
RocCurveDisplay, roc_auc_score, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import log_loss
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.svm import SVC

In [11]:
df = pd.read_csv('../Cases/Vehicle_Silhouettes/Vehicle.csv')
df

,Comp,Circ,D.Circ,Rad.Ra,Pr.Axis.Ra,Max.L.Ra,Scat.Ra,Elong,Pr.Axis.Rect,Max.L.Rect,Sc.Var.Maxis,Sc.Var.maxis,Ra.Gyr,Skew.Maxis,Skew.maxis,Kurt.maxis,Kurt.Maxis,Holl.Ra,Class
0,95,48,83,178,72,10,162,42,20,159,176,379,184,70,6,16,187,197,van
1,91,41,84,141,57,9,149,45,19,143,170,330,158,72,9,14,189,199,van
2,104,50,106,209,66,10,207,32,23,158,223,635,220,73,14,9,188,196,saab
3,93,41,82,159,63,9,144,46,19,143,160,309,127,63,6,10,199,207,van
4,85,44,70,205,103,52,149,45,19,144,241,325,188,127,9,11,180,183,bus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,93,39,87,183,64,8,169,40,20,134,200,422,149,72,7,25,188,195,saab
842,89,46,84,163,66,11,159,43,20,159,173,368,176,72,1,20,186,197,van
843,106,54,101,222,67,12,222,30,25,173,228,721,200,70,3,4,187,201,saab
844,86,36,78,146,58,7,135,50,18,124,155,270,148,66,0,25,190,195,saab


In [12]:
x,y = df.drop('Class', axis = 1), df['Class']

In [13]:
x_train, x_test, y_train,y_test = train_test_split(x,y,random_state=25, test_size=0.3, stratify=y)

Without scaling

In [14]:
svm = SVC(kernel='linear', decision_function_shape='ovo')
svm.fit(x_train,y_train)
y_pred = svm.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.8464566929133859


With Standard Scaling

In [15]:
scaler = StandardScaler()
pipe = Pipeline([('SCALING', scaler), ('SVM', svm)])
pipe.fit(x_train,y_train)
y_pred = pipe.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.8188976377952756


With MinMax Scaling

In [16]:
scaler = MinMaxScaler()
pipe = Pipeline([('SCALING', scaler), ('SVM', svm)])
pipe.fit(x_train,y_train)
y_pred = pipe.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.7480314960629921


using all three scales at one

In [17]:
scales = [StandardScaler(), MinMaxScaler(), None]

for scaler in scales:
    pipe = Pipeline([('SCALING', scaler), ('SVM', svm)])
    pipe.fit(x_train,y_train)
    y_pred = pipe.predict(x_test)
    print(f'{scaler}: {accuracy_score(y_test, y_pred)}')

StandardScaler(): 0.8188976377952756
MinMaxScaler(): 0.7480314960629921
None: 0.8464566929133859


In [19]:
scaler = StandardScaler()
dfs = ['ovo', 'ovr']
scores = []
c_value = np.linspace(0.01, 5, 15)
scores = []
pipe = Pipeline([('SCL', scaler), ("SVM", svm)])
for decision in tqdm(dfs):
    for c in (c_value):
        svm = SVC(kernel = 'linear', C=c, decision_function_shape=decision)
        pipe = Pipeline([('SCL', scaler), ("SVM", svm)])

        pipe.fit(x_train, y_train)
        y_pred = pipe.predict(x_test)
        scores.append([c,decision ,accuracy_score(y_test,y_pred)])

scores_df = pd.DataFrame(data = scores, columns = ['C Value','Decision' ,'accuracy Score'])
scores_df.sort_values('accuracy Score', ascending=False)

100%|██████████| 2/2 [00:00<00:00,  2.79it/s]


,C Value,Decision,accuracy Score
12,4.287143,ovo,0.842520
27,4.287143,ovr,0.842520
8,2.861429,ovo,0.838583
11,3.930714,ovo,0.838583
19,1.435714,ovr,0.838583
4,1.435714,ovo,0.838583
26,3.930714,ovr,0.838583
23,2.861429,ovr,0.838583
13,4.643571,ovo,0.834646
22,2.505000,ovr,0.834646


## using poly function and changing degree

In [20]:
degree = [2,3,4]

scaler = StandardScaler()
dfs = ['ovo', 'ovr']
scores = []
c_value = np.linspace(0.01, 5, 15)
scores = []
pipe = Pipeline([('SCL', scaler), ("SVM", svm)])
for decision in tqdm(dfs):
    for d in degree:
        for c in (c_value):
            svm = SVC(kernel = 'poly',degree = d, C=c, decision_function_shape=decision)
            pipe = Pipeline([('SCL', scaler), ("SVM", svm)])

            pipe.fit(x_train, y_train)
            y_pred = pipe.predict(x_test)
            scores.append([c,decision,d ,accuracy_score(y_test,y_pred)])

scores_df = pd.DataFrame(data = scores, columns = ['C Value','Decision','Degree' ,'accuracy Score'])
scores_df.sort_values('accuracy Score', ascending=False)

100%|██████████| 2/2 [00:02<00:00,  1.04s/it]


,C Value,Decision,Degree,accuracy Score
74,5.000000,ovr,3,0.791339
29,5.000000,ovo,3,0.791339
28,4.643571,ovo,3,0.783465
73,4.643571,ovr,3,0.783465
72,4.287143,ovr,3,0.767717
...,...,...,...,...
30,0.010000,ovo,4,0.381890
45,0.010000,ovr,2,0.377953
0,0.010000,ovo,2,0.377953
15,0.010000,ovo,3,0.346457
